In [1]:
#!pip install oandapyV20

import oandapyV20
from oandapyV20 import API
import oandapyV20.endpoints.pricing as pricing
import oandapyV20.endpoints.accounts as accounts
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.trades as trades 
import oandapyV20.endpoints.positions as positions
import oandapyV20.endpoints.transactions as trans
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.definitions.primitives as primitives
import pandas as pd
from dateutil.relativedelta import relativedelta
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote_plus
import psycopg2
import os
import io
from io import StringIO 
import boto3
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
import time 
import pygsheets
import datedelta
import calendar
import  csv
import json

from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream
import numpy as np
import datetime
from scipy import stats
def convert_to_df(instrument,granularity = "H1",count = 5000,percentile_test = 97,
                  test_col = 'delta_percentile_prev_wick',positive = True,start = date(2020,1,1),end = date(2020,1,10)):
    print()
    print()
    print()
    print(instrument,granularity,count,percentile_test)
    print('START:',start,' END:',end)
    if positive:
        negative = False
    else:
        negative = True
    params = {#"count":count,
              "granularity": granularity,
              "from": str(start.isoformat('T')),
              "to": str(end.isoformat('T'))
                 }
    print('PARAMS',params)
    print(instrument,type(instrument))
    r = instruments.InstrumentsCandles(instrument=instrument,
                                      params=params)
    client.request(r)
    dct = r.response
    lst = dct['candles']
    if len(lst) == 0:
        print('NO RESULT RETRUNING EMPTY DF...')
        return pd.DataFrame([0])
    lst = []
    i = 0
    
    for candle in dct['candles']:
        #IF CANDLE IS BULLISH
        if (float(candle['mid']['c']) - float(candle['mid']['o'])) > 0:
            max_vector = (float(candle['mid']['h']) - float(candle['mid']['l']))
            
        #IF CANDLE IS BEARISH
        else:
            max_vector = -1 * (float(candle['mid']['h']) - float(candle['mid']['l']))
            
        if i == 0:
            prev_delta = 0 
            prev_delta_max = 0
            prev_delta_vector = 0 
            prev_wick = 0
        else:
            prev_delta = abs(float(prev_candle['mid']['c']) - float(prev_candle['mid']['o']))
            prev_delta_max = abs(float(prev_candle['mid']['l']) - float(prev_candle['mid']['h']))
            prev_delta_vector = (float(prev_candle['mid']['c']) - float(prev_candle['mid']['o']))
            
            if (float(prev_candle['mid']['c']) - float(prev_candle['mid']['o'])) > 0:
                prev_wick = float(prev_candle['mid']['h']) - float(prev_candle['mid']['c'])
            else:
                prev_wick = float(prev_candle['mid']['l']) - float(prev_candle['mid']['o'])


        lst.append([candle['time'],candle['volume'],float(candle['mid']['o']),float(candle['mid']['h']),float(candle['mid']['l']),
                        float(candle['mid']['c']),abs(float(candle['mid']['c']) - float(candle['mid']['o'])),
                    abs(float(candle['mid']['l']) - float(candle['mid']['h'])),
                     (float(candle['mid']['c']) - float(candle['mid']['o'])),max_vector,

                    prev_delta,prev_delta_max,prev_delta_vector,prev_wick

                   ])
        
        prev_candle = candle
        i += 1


    df = pd.DataFrame(lst,columns = ['time','volume','o','h','l','c','delta','delta_max',
                                     'delta_vector','max_vector','prev_delta',
                                     'prev_delta_max','prev_delta_vector','prev_wick'])
    
    
    df['prev_5'] = 0
    df['prev_10'] = 0
    df['prev_24'] = 0
    
    
    df['next_candle'] = 0
    df['next_2'] = 0
    df['next_3'] = 0
    df['next_4'] = 0
    df['next_5'] = 0
    df['next_10'] = 0
    
    df['next_candle+1'] = 0
    df['next_2+1'] = 0
    df['next_3+1'] = 0
    df['next_4+1'] = 0
    df['next_5+1'] = 0
    df['next_10+1'] = 0
    df['next_20+1'] = 0
        
    tot_length = df.shape[0]
    for i in range(df.shape[0] - 21):
        
        df['prev_5'].iloc[i] = df['c'].iloc[i] - df['c'].iloc[i - 5] 
        df['prev_10'].iloc[i] = df['c'].iloc[i] - df['c'].iloc[i - 10] 
        df['prev_24'].iloc[i] = df['c'].iloc[i] - df['c'].iloc[i - 24]    
        
        df['next_candle'].iloc[i] = df['c'].iloc[i + 1] - df['c'].iloc[i]
        df['next_2'].iloc[i] = df['c'].iloc[i + 2] - df['c'].iloc[i]
        df['next_3'].iloc[i] = df['c'].iloc[i + 3] - df['c'].iloc[i]
        df['next_4'].iloc[i] = df['c'].iloc[i + 4] - df['c'].iloc[i]
        df['next_5'].iloc[i] = df['c'].iloc[i + 5] - df['c'].iloc[i]
        df['next_10'].iloc[i] = df['c'].iloc[i + 10] - df['c'].iloc[i]
        
        df['next_candle+1'].iloc[i] = df['c'].iloc[i + 1+1] - df['c'].iloc[i+1]
        df['next_2+1'].iloc[i] = df['c'].iloc[i + 2+1] - df['c'].iloc[i+1]
        df['next_3+1'].iloc[i] = df['c'].iloc[i + 3+1] - df['c'].iloc[i+1]
        df['next_4+1'].iloc[i] = df['c'].iloc[i + 4+1] - df['c'].iloc[i+1]
        df['next_5+1'].iloc[i] = df['c'].iloc[i + 5+1] - df['c'].iloc[i+1]
        df['next_10+1'].iloc[i] = df['c'].iloc[i + 10+1] - df['c'].iloc[i+1]
        df['next_20+1'].iloc[i] = df['c'].iloc[i + 20+1] - df['c'].iloc[i+1] 
        
        
    col = test_col
    filter_df = df[df[col] >= np.percentile(df[col], percentile_test)]
    print('SHAPE',filter_df.shape)
    df['trigger'] = 0

    df.loc[df[df[col] >= np.percentile(df[col], percentile_test)].index,'trigger'] = 1    
    
    print('SHAPE2',df[df.trigger == 1].shape)
        
    return df

def convert_to_simple_df(instrument,granularity = "H1",count = 5000,start = date(2020,1,1),end = date(2020,1,10)):
    print()
    print()
    print()
    print(instrument,granularity,count)
    print('START:',start,' END:',end)

    params = {#"count":count,
              "granularity": granularity,
              "from": str(start.isoformat('T')),
              "to": str(end.isoformat('T'))
                 }
    print('PARAMS',params)
    print(instrument,type(instrument))
    r = instruments.InstrumentsCandles(instrument=instrument,
                                      params=params)
    client.request(r)
    dct = r.response
    lst = dct['candles']
    if len(lst) == 0:
        print('NO RESULT RETRUNING EMPTY DF...')
        return pd.DataFrame([0])
    lst = []
    i = 0
    
    for candle in dct['candles']:

        lst.append([candle['time'],
                    candle['volume'],
                    float(candle['mid']['o']),
                    float(candle['mid']['h']),
                    float(candle['mid']['l']),
                    float(candle['mid']['c'])
                   ])
        
 
        i += 1


    df = pd.DataFrame(lst,columns = ['time','volume','o','h','l','c'])
    
    df['utc_timestamp'] = pd.to_datetime(df['time'], format = '%Y-%m-%d %H:%M:%S', errors ='coerce')
    df['est_timestamp'] = df['utc_timestamp'].dt.tz_convert('US/Eastern')        
    return df
accountID='101-001-20062555-002'
access_token='2ad78612c58e604890fd961550f73cfd-28190cc09d5cbc67b2c14503575d6132'

api = API(access_token)
client=API(access_token)

jpy_pairs=['GBP_JPY','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY']
major_pairs=['GBP_USD','USD_CAD','USD_CHF','EUR_USD','AUD_USD','NZD_USD']
cross_pairs=['AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF','EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF']
all_pairs=['USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
           'GBP_USD','USD_CAD','USD_CHF','EUR_USD','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF']





# CREATE AND SAVE M1

In [ ]:
def convert_to_simple_df(instrument,granularity = "H1",count = 5000,start = date(2020,1,1),end = date(2020,1,10)):
    print()
    print()
    print()
    print(instrument,granularity,count)
    print('START:',start,' END:',end)

    params = {#"count":count,
              "granularity": granularity,
              "from": str(start.isoformat('T')),
              "to": str(end.isoformat('T'))
                 }
    print('PARAMS',params)
    print(instrument,type(instrument))
    r = instruments.InstrumentsCandles(instrument=instrument,
                                      params=params)
    client.request(r)
    dct = r.response
    lst = dct['candles']
    if len(lst) == 0:
        print('NO RESULT RETRUNING EMPTY DF...')
        return pd.DataFrame([0])
    lst = []
    i = 0
    
    for candle in dct['candles']:

        lst.append([candle['time'],
                    candle['volume'],
                    float(candle['mid']['o']),
                    float(candle['mid']['h']),
                    float(candle['mid']['l']),
                    float(candle['mid']['c'])
                   ])
        
 
        i += 1


    df = pd.DataFrame(lst,columns = ['time','volume','o','h','l','c'])
    
    df['utc_timestamp'] = pd.to_datetime(df['time'], format = '%Y-%m-%d %H:%M:%S', errors ='coerce')
    df['est_timestamp'] = df['utc_timestamp'].dt.tz_convert('US/Eastern')        
    return df


def create_data_and_save(instrument,start,end,pair,hours = 3,granularity = "M1"):

    print('CREATING DATASET...')
    s = time.time()
    print('PARAMS:',start,end,pair,granularity)
    cur = start

    #i = 'GBP_JPY'
    total_df = 0
    dct = {}
    
    dir_name = os.getcwd() + '/' + pair + '_' + granularity
    
    isdir = os.path.isdir(dir_name) 
    if isdir:
        pass
    else:
        print('MAKING DIRECTORY:',dir_name)
        os.mkdir(dir_name) 
    path = dir_name + '/' + str(date(start.year,start.month,start.day)) + '_' + str(date(end.year,end.month,end.day)) + '.csv'
    print('PATH:',path)
    while cur < end:
        print(cur)    
        df = convert_to_simple_df(instrument = instrument,granularity = granularity,count = 5000,start = cur,end = cur + timedelta(hours = hours))
        

        cur = cur + timedelta(hours = hours)

        print(df.shape)
        if df.shape[1] == 1:
            pass
        else:
            try:
                total_df = total_df.append(df)
            except:
                total_df = df
                
            dct[str(cur)] = df

    print(total_df.shape)
    total_df.to_csv(path,index = False)
    e = time.time()
    print('TOTAL DATASET CONSTRUCTION TIME:',(e-s)/60,' MINUTES')
accountID='101-001-20062555-002'
access_token='2ad78612c58e604890fd961550f73cfd-28190cc09d5cbc67b2c14503575d6132'

api = API(access_token)
client=API(access_token)

jpy_pairs=['GBP_JPY','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY']
major_pairs=['GBP_USD','USD_CAD','USD_CHF','EUR_USD','AUD_USD','NZD_USD']
cross_pairs=['AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF','EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF']
all_pairs=['USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
           'GBP_USD','USD_CAD','USD_CHF','EUR_USD','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF']    
jpy_pairs=['GBP_JPY','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY']
major_pairs=['GBP_USD','USD_CAD','USD_CHF','EUR_USD','AUD_USD','NZD_USD']
cross_pairs=['AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF','EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF']
all_pairs=['XAU_USD','GBP_JPY','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
           'GBP_USD','USD_CAD','USD_CHF','EUR_USD','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 


all_pairs=['GBP_JPY','EUR_USD','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
           'GBP_USD','USD_CAD','USD_CHF','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 


all_pairs=['AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 

start = datetime.datetime(2016,1,1,0,0,0)
end = datetime.datetime(2022,7,31,0,0,0)
for pair in all_pairs:
    create_data_and_save(instrument = pair,start = start,end = end,pair = pair,hours = 60,granularity = "M1")

        
        

In [ ]:
all_pairs=['USD_CHF','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 

# Create and Save M5

In [2]:
def convert_to_simple_df(instrument,granularity = "H1",count = 5000,start = date(2020,1,1),end = date(2020,1,10)):
    print()
    print()
    print()
    print(instrument,granularity,count)
    print('START:',start,' END:',end)

    params = {#"count":count,
              "granularity": granularity,
              "from": str(start.isoformat('T')),
              "to": str(end.isoformat('T'))
                 }
    print('PARAMS',params)
    print(instrument,type(instrument))
    r = instruments.InstrumentsCandles(instrument=instrument,
                                      params=params)
    client.request(r)
    dct = r.response
    lst = dct['candles']
    if len(lst) == 0:
        print('NO RESULT RETRUNING EMPTY DF...')
        return pd.DataFrame([0])
    lst = []
    i = 0
    
    for candle in dct['candles']:

        lst.append([candle['time'],
                    candle['volume'],
                    float(candle['mid']['o']),
                    float(candle['mid']['h']),
                    float(candle['mid']['l']),
                    float(candle['mid']['c'])
                   ])
        
 
        i += 1


    df = pd.DataFrame(lst,columns = ['time','volume','o','h','l','c'])
    
    df['utc_timestamp'] = pd.to_datetime(df['time'], format = '%Y-%m-%d %H:%M:%S', errors ='coerce')
    df['est_timestamp'] = df['utc_timestamp'].dt.tz_convert('US/Eastern')        
    return df


def create_data_and_save(instrument,start,end,pair,hours = 3,granularity = "M1"):

    print('CREATING DATASET...')
    s = time.time()
    print('PARAMS:',start,end,pair,granularity)
    cur = start

    #i = 'GBP_JPY'
    total_df = 0
    dct = {}
    
    dir_name = os.getcwd() + '/' + pair + '_' + granularity
    
    isdir = os.path.isdir(dir_name) 
    if isdir:
        pass
    else:
        print('MAKING DIRECTORY:',dir_name)
        os.mkdir(dir_name) 
    path = dir_name + '/' + str(date(start.year,start.month,start.day)) + '_' + str(date(end.year,end.month,end.day)) + '.csv'
    print('PATH:',path)
    while cur < end:
        print(cur)    
        df = convert_to_simple_df(instrument = instrument,granularity = granularity,count = 5000,start = cur,end = cur + timedelta(hours = hours))
        

        cur = cur + timedelta(hours = hours)

        print(df.shape)
        if df.shape[1] == 1:
            pass
        else:
            try:
                total_df = total_df.append(df)
            except:
                total_df = df
                
            dct[str(cur)] = df

    print(total_df.shape)
    total_df.to_csv(path,index = False)
    e = time.time()
    print('TOTAL DATASET CONSTRUCTION TIME:',(e-s)/60,' MINUTES')
accountID='101-001-20062555-002'
access_token='2ad78612c58e604890fd961550f73cfd-28190cc09d5cbc67b2c14503575d6132'

api = API(access_token)
client=API(access_token)

jpy_pairs=['GBP_JPY','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY']
major_pairs=['GBP_USD','USD_CAD','USD_CHF','EUR_USD','AUD_USD','NZD_USD']
cross_pairs=['AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF','EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF']
all_pairs=['USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
           'GBP_USD','USD_CAD','USD_CHF','EUR_USD','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF']    
jpy_pairs=['GBP_JPY','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY']
major_pairs=['GBP_USD','USD_CAD','USD_CHF','EUR_USD','AUD_USD','NZD_USD']
cross_pairs=['AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF','EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF']
all_pairs=['XAU_USD','GBP_JPY','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
           'GBP_USD','USD_CAD','USD_CHF','EUR_USD','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 


all_pairs=['GBP_JPY','EUR_USD','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
           'GBP_USD','USD_CAD','USD_CHF','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 
start = datetime.datetime(2010,1,1,0,0,0)
end = datetime.datetime(2016,1,1,0,0,0)

for pair in all_pairs:
    create_data_and_save(instrument = pair,start = start,end = end,pair = pair,hours = 250,granularity = "M5")

        
        

CREATING DATASET...
PARAMS: 2010-01-01 00:00:00 2016-01-01 00:00:00 GBP_JPY M5
PATH: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/GBP_JPY_M5/2010-01-01_2016-01-01.csv
2010-01-01 00:00:00



GBP_JPY M5 5000
START: 2010-01-01 00:00:00  END: 2010-01-11 10:00:00
PARAMS {'granularity': 'M5', 'from': '2010-01-01T00:00:00', 'to': '2010-01-11T10:00:00'}
GBP_JPY <class 'str'>
(1736, 8)
2010-01-11 10:00:00



GBP_JPY M5 5000
START: 2010-01-11 10:00:00  END: 2010-01-21 20:00:00
PARAMS {'granularity': 'M5', 'from': '2010-01-11T10:00:00', 'to': '2010-01-21T20:00:00'}
GBP_JPY <class 'str'>
(2473, 8)
2010-01-21 20:00:00



GBP_JPY M5 5000
START: 2010-01-21 20:00:00  END: 2010-02-01 06:00:00
PARAMS {'granularity': 'M5', 'from': '2010-01-21T20:00:00', 'to': '2010-02-01T06:00:00'}
GBP_JPY <class 'str'>


<ipython-input-2-93d6094133cc>:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_df = total_df.append(df)


(1947, 8)
2010-02-01 06:00:00



GBP_JPY M5 5000
START: 2010-02-01 06:00:00  END: 2010-02-11 16:00:00
PARAMS {'granularity': 'M5', 'from': '2010-02-01T06:00:00', 'to': '2010-02-11T16:00:00'}
GBP_JPY <class 'str'>


<ipython-input-2-93d6094133cc>:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_df = total_df.append(df)


(2471, 8)
2010-02-11 16:00:00



GBP_JPY M5 5000
START: 2010-02-11 16:00:00  END: 2010-02-22 02:00:00
PARAMS {'granularity': 'M5', 'from': '2010-02-11T16:00:00', 'to': '2010-02-22T02:00:00'}
GBP_JPY <class 'str'>


...

(2424, 8)
2015-12-18 02:00:00



NZD_CHF M5 5000
START: 2015-12-18 02:00:00  END: 2015-12-28 12:00:00
PARAMS {'granularity': 'M5', 'from': '2015-12-18T02:00:00', 'to': '2015-12-28T12:00:00'}
NZD_CHF <class 'str'>


<ipython-input-2-93d6094133cc>:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_df = total_df.append(df)


(1524, 8)
2015-12-28 12:00:00



NZD_CHF M5 5000
START: 2015-12-28 12:00:00  END: 2016-01-07 22:00:00
PARAMS {'granularity': 'M5', 'from': '2015-12-28T12:00:00', 'to': '2016-01-07T22:00:00'}
NZD_CHF <class 'str'>
(2098, 8)
(454453, 8)


<ipython-input-2-93d6094133cc>:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_df = total_df.append(df)
<ipython-input-2-93d6094133cc>:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_df = total_df.append(df)


TOTAL DATASET CONSTRUCTION TIME: 1.1426653146743775  MINUTES


In [19]:
180 / 24

7.5

In [21]:
5000 / (60 /5)

416.6666666666667

In [14]:
17561355 + 5853786

23415141

# Create and save M15

In [ ]:
start = date(2020,1,1)
end = date(2022,1,1)
pair = 'XAU_USD'
def create_data_and_save(start,end,pair,hours = 3,granularity = "M1"):

    print('CREATING DATASET...')
    s = time.time()
    print('PARAMS:',start,end,pair,granularity)
    cur = start

    #i = 'GBP_JPY'
    total_df = 0
    dct = {}
    
    dir_name = os.getcwd() + '/' + pair + '_' + granularity
    
    isdir = os.path.isdir(dir_name) 
    if isdir:
        pass
    else:
        print('MAKING DIRECTORY:',dir_name)
        os.mkdir(dir_name) 
    path = dir_name + '/' + str(date(start.year,start.month,start.day)) + '_' + str(date(end.year,end.month,end.day)) + '.csv'
    print('PATH:',path)
    while cur < end:
        print(cur)    
        df = convert_to_df(instrument = pair,granularity = granularity,count = 5000,
                           percentile_test = 99.3,test_col = 'prev_delta',positive = True,start = cur,end = cur + timedelta(hours = hours))
        cur = cur + timedelta(hours = hours)

        print(df.shape)
        if df.shape[1] == 1:
            pass
        else:
            try:
                total_df = total_df.append(df)
            except:
                total_df = df
                
            dct[str(cur)] = df

    print(total_df.shape)
    total_df.to_csv(path,index = False)
    e = time.time()
    print('TOTAL DATASET CONSTRUCTION TIME:',(e-s)/60,' MINUTES')
    
jpy_pairs=['GBP_JPY','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY']
major_pairs=['GBP_USD','USD_CAD','USD_CHF','EUR_USD','AUD_USD','NZD_USD']
cross_pairs=['AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF','EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF']
all_pairs=['XAU_USD','GBP_JPY','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
           'GBP_USD','USD_CAD','USD_CHF','EUR_USD','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 
all_pairs = ['XAU_USD','GBP_JPY','USD_JPY','EUR_USD']
all_pairs = ['GBP_JPY','USD_JPY','EUR_USD']
start = datetime.datetime(2019,1,1,0,0,0)
end = datetime.datetime(2022,1,31,0,0,0)
for pair in all_pairs:
    create_data_and_save(start = start,end = end,pair = pair,hours = 180,granularity = "M15")

        
        

In [10]:
(60 / 5) * 15

180.0

# Create and Save Dataset S5

In [ ]:
start = date(2020,1,1)
end = date(2022,1,1)
pair = 'XAU_USD'
def create_data_and_save(start,end,pair,hours = 3,granularity = "M1"):

    print('CREATING DATASET...')
    s = time.time()
    print('PARAMS:',start,end,pair,granularity)
    cur = start

    #i = 'GBP_JPY'
    total_df = 0
    dct = {}
    
    dir_name = os.getcwd() + '/' + pair + '_' + granularity
    
    isdir = os.path.isdir(dir_name) 
    if isdir:
        pass
    else:
        print('MAKING DIRECTORY:',dir_name)
        os.mkdir(dir_name) 
    path = dir_name + '/' + str(date(start.year,start.month,start.day)) + '_' + str(date(end.year,end.month,end.day)) + '.csv'
    print('PATH:',path)
    while cur < end:
        time.sleep(1.5)
        print(cur)    
        df = convert_to_df(instrument = pair,granularity = granularity,count = 5000,
                           percentile_test = 99.3,test_col = 'prev_delta',positive = True,start = cur,end = cur + timedelta(hours = hours))
        cur = cur + timedelta(hours = hours)

        print(df.shape)
        if df.shape[1] == 1:
            pass
        else:
            try:
                total_df = total_df.append(df)
            except:
                total_df = df
            dct[str(cur)] = df

    print(total_df.shape)
    total_df.to_csv(path,index = False)
    e = time.time()
    print('TOTAL DATASET CONSTRUCTION TIME:',(e-s)/60,' MINUTES')
    
jpy_pairs=['GBP_JPY','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY']
major_pairs=['GBP_USD','USD_CAD','USD_CHF','EUR_USD','AUD_USD','NZD_USD']
cross_pairs=['AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF','EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF']
all_pairs=['XAU_USD','GBP_JPY','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
           'GBP_USD','USD_CAD','USD_CHF','EUR_USD','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 
all_pairs = ['XAU_USD','GBP_JPY','USD_JPY','EUR_USD']
all_pairs = ['USD_JPY','EUR_USD']
start = datetime.datetime(2021,12,1,0,0,0)
end = datetime.datetime(2022,1,31,0,0,0)

for pair in all_pairs:
    print(pair)
    start_ = start
    end_ = start_ + relativedelta(months=1)
    while end_ < end:
        print('RUNNING FOR MONTH',start_,end_)
        create_data_and_save(start = start_,end = end_,pair = pair,hours = 5,granularity = "S5")
        start_ = start_ + relativedelta(months=1)
        end_ = start_ + relativedelta(months=1)
        print('FINISHED WITH MONTH')
        print()
        print()
        print()
        print()
        print()

        
        

In [ ]:
2020-11-01 00:00:00 2020-12-01 00:00:00

In [ ]:
from dateutil.relativedelta import relativedelta
datetime.datetime(2020,1,1,0,0,0) + relativedelta(months=1)

In [ ]:
datetime.datetime + relativedelta(months=n)

In [ ]:
df = pd.DataFrame([0])
df.to_csv('test/test.csv')

In [ ]:
isdir = os.path.isdir(os.getcwd() + '/test') 
print(isdir) 

In [ ]:
os.getcwd()

In [ ]:
def convert_to_df(instrument,granularity = "H1",count = 5000,percentile_test = 97,
                  test_col = 'delta_percentile_prev_wick',positive = True,start = date(2020,1,1),end = date(2020,1,10)):
    print()
    print()
    print()
    print(instrument,granularity,count,percentile_test)
    print('START:',start,' END:',end)
    if positive:
        negative = False
    else:
        negative = True
    params = {#"count":count,
              "granularity": granularity,
              "from": start,
              "to":end
             }
    print(instrument,type(instrument))
    r = instruments.InstrumentsCandles(instrument=instrument,
                                      params=params)
    client.request(r)
    dct = r.response
    lst = []
    i = 0
    
    for candle in dct['candles']:
        #IF CANDLE IS BULLISH
        if (float(candle['mid']['c']) - float(candle['mid']['o'])) > 0:
            max_vector = (float(candle['mid']['h']) - float(candle['mid']['l']))
            
        #IF CANDLE IS BEARISH
        else:
            max_vector = -1 * (float(candle['mid']['h']) - float(candle['mid']['l']))
            
        if i == 0:
            prev_delta = 0 
            prev_delta_max = 0
            prev_delta_vector = 0 
            prev_wick = 0
        else:
            prev_delta = abs(float(prev_candle['mid']['c']) - float(prev_candle['mid']['o']))
            prev_delta_max = abs(float(prev_candle['mid']['l']) - float(prev_candle['mid']['h']))
            prev_delta_vector = (float(prev_candle['mid']['c']) - float(prev_candle['mid']['o']))
            
            if (float(prev_candle['mid']['c']) - float(prev_candle['mid']['o'])) > 0:
                prev_wick = float(prev_candle['mid']['h']) - float(prev_candle['mid']['c'])
            else:
                prev_wick = float(prev_candle['mid']['l']) - float(prev_candle['mid']['o'])


        lst.append([candle['time'],candle['volume'],float(candle['mid']['o']),float(candle['mid']['h']),float(candle['mid']['l']),
                        float(candle['mid']['c']),abs(float(candle['mid']['c']) - float(candle['mid']['o'])),
                    abs(float(candle['mid']['l']) - float(candle['mid']['h'])),
                     (float(candle['mid']['c']) - float(candle['mid']['o'])),max_vector,

                    prev_delta,prev_delta_max,prev_delta_vector,prev_wick

                   ])
        
        prev_candle = candle
        i += 1


    df = pd.DataFrame(lst,columns = ['time','volume','o','h','l','c','delta','delta_max',
                                     'delta_vector','max_vector','prev_delta',
                                     'prev_delta_max','prev_delta_vector','prev_wick'])
    
    
    df['prev_5'] = 0
    df['prev_10'] = 0
    df['prev_24'] = 0
    
    
    df['next_candle'] = 0
    df['next_2'] = 0
    df['next_3'] = 0
    df['next_4'] = 0
    df['next_5'] = 0
    df['next_10'] = 0
    
    df['next_candle+1'] = 0
    df['next_2+1'] = 0
    df['next_3+1'] = 0
    df['next_4+1'] = 0
    df['next_5+1'] = 0
    df['next_10+1'] = 0
    df['next_20+1'] = 0
        
    tot_length = df.shape[0]
    for i in range(df.shape[0] - 21):
        
        df['prev_5'].iloc[i] = df['c'].iloc[i] - df['c'].iloc[i - 5] 
        df['prev_10'].iloc[i] = df['c'].iloc[i] - df['c'].iloc[i - 10] 
        df['prev_24'].iloc[i] = df['c'].iloc[i] - df['c'].iloc[i - 24]    
        
        df['next_candle'].iloc[i] = df['c'].iloc[i + 1] - df['c'].iloc[i]
        df['next_2'].iloc[i] = df['c'].iloc[i + 2] - df['c'].iloc[i]
        df['next_3'].iloc[i] = df['c'].iloc[i + 3] - df['c'].iloc[i]
        df['next_4'].iloc[i] = df['c'].iloc[i + 4] - df['c'].iloc[i]
        df['next_5'].iloc[i] = df['c'].iloc[i + 5] - df['c'].iloc[i]
        df['next_10'].iloc[i] = df['c'].iloc[i + 10] - df['c'].iloc[i]
        
        df['next_candle+1'].iloc[i] = df['c'].iloc[i + 1+1] - df['c'].iloc[i+1]
        df['next_2+1'].iloc[i] = df['c'].iloc[i + 2+1] - df['c'].iloc[i+1]
        df['next_3+1'].iloc[i] = df['c'].iloc[i + 3+1] - df['c'].iloc[i+1]
        df['next_4+1'].iloc[i] = df['c'].iloc[i + 4+1] - df['c'].iloc[i+1]
        df['next_5+1'].iloc[i] = df['c'].iloc[i + 5+1] - df['c'].iloc[i+1]
        df['next_10+1'].iloc[i] = df['c'].iloc[i + 10+1] - df['c'].iloc[i+1]
        df['next_20+1'].iloc[i] = df['c'].iloc[i + 20+1] - df['c'].iloc[i+1] 
        
        
    col = test_col
    filter_df = df[df[col] >= np.percentile(df[col], percentile_test)]
    print('SHAPE',filter_df.shape)
    df['trigger'] = 0

    df.loc[df[df[col] >= np.percentile(df[col], percentile_test)].index,'trigger'] = 1    
    
    print('SHAPE2',df[df.trigger == 1].shape)
        
    return df
start = date(2021,1,1)
end = date(2022,1,1)

cur = start
pair = 'XAU_USD'
#i = 'GBP_JPY'
total_df = 0
dct = {}
days = 3
granularity
path = pair + '_M1_' + str(start) + '_' + str(end) + '.csv'
while cur < end:
    
    print(cur)    
    df = convert_to_df(instrument = pair,granularity = "M1",count = 5000,
                       percentile_test = 99.9,test_col = 'prev_delta',positive = True,start = cur,end = cur + timedelta(days = days))
    cur = cur + timedelta(days = days)
    
    print(df.shape)
    try:
        total_df = total_df.append(df)
    except:
        total_df = df
    dct[str(cur)] = df
    
print(total_df.shape)
total_df.to_csv(path,index = False)


In [ ]:
start = date(2020,1,1)
end = date(2022,1,1)
pair = 'XAU_USD'
def create_data_and_save(start,end,pair,days = 3,granularity = "M1"):

    print('CREATING DATASET...')
    s = time.time()
    print('PARAMS:',start,end,pair,granularity)
    cur = start

    #i = 'GBP_JPY'
    total_df = 0
    dct = {}

    path = pair + '_' + granularity + '_' + str(start) + '_' + str(end) + '.csv'
    print('PATH:',path)
    while cur < end:

        print(cur)    
        df = convert_to_df(instrument = pair,granularity = "granularity",count = 5000,
                           percentile_test = 99.3,test_col = 'prev_delta',positive = True,start = cur,end = cur + timedelta(days = days))
        cur = cur + timedelta(days = days)

        print(df.shape)
        try:
            total_df = total_df.append(df)
        except:
            total_df = df
        dct[str(cur)] = df

    print(total_df.shape)
    total_df.to_csv(path,index = False)
    e = time.time()
    print('TOTAL DATASET CONSTRUCTION TIME:',(e-s)/60,' MINUTES')
    

jpy_pairs=['GBP_JPY','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY']
major_pairs=['GBP_USD','USD_CAD','USD_CHF','EUR_USD','AUD_USD','NZD_USD']
cross_pairs=['AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF','EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF']
all_pairs=['XAU_USD','GBP_JPY','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
           'GBP_USD','USD_CAD','USD_CHF','EUR_USD','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 
all_pairs = ['XAU_USD','GBP_JPY','USD_JPY','EUR_USD']
for pair in all_pairs:
    print(pair)
    create_data_and_save(start = date(2019,1,1),end = date(2022,1,31),pair = pair,days = 3,granularity = "M1")
    


In [ ]:
total_df.to_csv(path,index = False)

In [ ]:
os.getcwd()

In [ ]:
total_df.head()

In [ ]:
len(dct_keys)

In [ ]:
i = df[df.trigger == 1].index[0]
i

In [ ]:
import random
dct_keys = list(dct.keys())
rand = random.randint(0, len(dct_keys) - 1)
col = 'prev_delta'
#df = total_df.copy()
df = dct[dct_keys[rand]]
print(df[df.trigger == 1].shape)
rand2 = random.randint(0, df[df.trigger == 1].shape[0] - 1)
i = df[df.trigger == 1].index[rand2]

window = 125

fig = go.Figure(data=go.Candlestick(x=df.iloc[i-window:i + window,:]['time'],
                    open=df.iloc[i-window:i + window,:]['o'],
                    high=df.iloc[i-window:i + window,:]['h'],
                    low=df.iloc[i-window:i + window,:]['l'],
                    close=df.iloc[i-window:i + window,:]['c']))


fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i], 
              annotation_text="trigger point " + str(round(df[col].iloc[i],4)) + ' NEXT:'+ str(round(df[col].iloc[i + 1],4)), annotation_position="top left",
              fillcolor="green", opacity=0.25, line_width=0)


fig.show()

In [ ]:
col = 'prev_delta'
print(df[df.trigger == 1].shape)
i = df[df.trigger == 1].index[52]


fig = go.Figure(data=go.Candlestick(x=df.iloc[i-20:i + 50,:]['time'],
                    open=df.iloc[i-20:i + 50,:]['o'],
                    high=df.iloc[i-20:i + 50,:]['h'],
                    low=df.iloc[i-20:i + 50,:]['l'],
                    close=df.iloc[i-20:i + 50,:]['c']))


fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i], 
              annotation_text="trigger point " + str(round(df[col].iloc[i],4)), annotation_position="top left",
              fillcolor="green", opacity=0.25, line_width=0)


fig.show()

In [ ]:
df.

In [ ]:
col = 'prev_delta'
df[df[col] >= np.percentile(df[col], 99.9)]

In [ ]:
    
def test_method(df,positive = True,test_col = 'delta_percentile_prev_wick',percentile_test = 97):
    print('TEST 2')
    if positive:
        negative = False
    else:
        negative = True
    x = list(df.delta)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile'] = delta_percentile


    x = list(df.prev_delta)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile_prev'] = delta_percentile


    x = list(df.prev_delta_vector)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile_prev_vector'] = delta_percentile

    
    x = list(df.prev_wick)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile_prev_wick'] = delta_percentile

    x = list(df['prev_24'])
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_prev_24'] = delta_percentile        
    
    print(df.shape)

    if positive:
        top_x = df[(df[test_col] >= percentile_test)]
    if negative:
        top_x = df[(df[test_col] < percentile_test)]
        
    print(top_x.shape)    
    
    pos = []
    neg = []
    for i in range(top_x.shape[0]):
        val = round(top_x.delta_vector.iloc[i],4)
        if val < 0:
            neg.append(val)
        else:
            pos.append(val)

    print('POSITIVE:',len(pos),sum(pos),np.mean(pos),np.median(pos))
    print('NEGATIVE:',len(neg),sum(neg),np.mean(neg),np.median(neg))  
    print('OVERALL:',sum(top_x.delta_vector),np.mean(top_x.delta_vector),np.median(top_x.delta_vector))
    print('NEXT CANDLE FUTURE OVERALL:',sum(top_x.next_candle),np.mean(top_x.next_candle),
              np.median(top_x.next_candle),'count neg:',top_x[top_x.next_candle < 0].shape[0],'count pos:',top_x[top_x.next_candle > 0].shape[0])
    
    col = 'next_2'
    print('2 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0])    
    col = 'next_3'
    print('3 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0])    
    
    col = 'next_4'
    print('4 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0])    
    col = 'next_5'
    print('5 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0]) 
    col = 'next_10'
    print('10 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0]) 
    return top_x

In [ ]:

df = total_df.copy()
top_x = test_method(df,positive = True,test_col = 'delta_percentile_prev_vector',percentile_test = 99.9)

dct[i] = top_x


df['trigger'] = 0

df.loc[df[df.time.isin(top_x.time)].index,'trigger'] = 1
print(df[df.trigger == 1].index[0])

In [ ]:
df = convert_to_df(instrument = i,granularity = "M5",count = 5000,
                   percentile_test = 97,test_col = 'delta_percentile_prev_wick',positive = True,start = date(2020,1,15),end = date(2020,1,16))
print(df.shape)
df.head()

In [ ]:
(60 * 24 * 30) / 5

In [ ]:
def convert_to_df(instrument,granularity = "H1",count = 5000,percentile_test = 97,test_col = 'delta_percentile_prev_wick',positive = True):
    print()
    print()
    print()
    print(instrument,granularity,count,percentile_test)
    if positive:
        negative = False
    else:
        negative = True
    params = {"count":count,
              "granularity": granularity
             }
    print(instrument,type(instrument))
    r = instruments.InstrumentsCandles(instrument=instrument,
                                      params=params)
    client.request(r)
    dct = r.response
    lst = []
    i = 0
    
    for candle in dct['candles']:
        #IF CANDLE IS BULLISH
        if (float(candle['mid']['c']) - float(candle['mid']['o'])) > 0:
            max_vector = (float(candle['mid']['h']) - float(candle['mid']['l']))
            
        #IF CANDLE IS BEARISH
        else:
            max_vector = -1 * (float(candle['mid']['h']) - float(candle['mid']['l']))
            
        if i == 0:
            prev_delta = 0 
            prev_delta_max = 0
            prev_delta_vector = 0 
            prev_wick = 0
        else:
            prev_delta = abs(float(prev_candle['mid']['c']) - float(prev_candle['mid']['o']))
            prev_delta_max = abs(float(prev_candle['mid']['l']) - float(prev_candle['mid']['h']))
            prev_delta_vector = (float(prev_candle['mid']['c']) - float(prev_candle['mid']['o']))
            
            if (float(prev_candle['mid']['c']) - float(prev_candle['mid']['o'])) > 0:
                prev_wick = float(prev_candle['mid']['h']) - float(prev_candle['mid']['c'])
            else:
                prev_wick = float(prev_candle['mid']['l']) - float(prev_candle['mid']['o'])


        lst.append([candle['time'],candle['volume'],float(candle['mid']['o']),float(candle['mid']['h']),float(candle['mid']['l']),
                        float(candle['mid']['c']),abs(float(candle['mid']['c']) - float(candle['mid']['o'])),
                    abs(float(candle['mid']['l']) - float(candle['mid']['h'])),
                     (float(candle['mid']['c']) - float(candle['mid']['o'])),max_vector,

                    prev_delta,prev_delta_max,prev_delta_vector,prev_wick

                   ])
        
        prev_candle = candle
        i += 1


    df = pd.DataFrame(lst,columns = ['time','volume','o','h','l','c','delta','delta_max',
                                     'delta_vector','max_vector','prev_delta',
                                     'prev_delta_max','prev_delta_vector','prev_wick'])
    
    
    df['prev_5'] = 0
    df['prev_10'] = 0
    df['prev_24'] = 0
    
    
    df['next_candle'] = 0
    df['next_2'] = 0
    df['next_3'] = 0
    df['next_4'] = 0
    df['next_5'] = 0
    df['next_10'] = 0
    
    df['next_candle+1'] = 0
    df['next_2+1'] = 0
    df['next_3+1'] = 0
    df['next_4+1'] = 0
    df['next_5+1'] = 0
    df['next_10+1'] = 0
    df['next_20+1'] = 0
        
    tot_length = df.shape[0]
    for i in range(df.shape[0] - 21):
        
        df['prev_5'].iloc[i] = df['c'].iloc[i] - df['c'].iloc[i - 5] 
        df['prev_10'].iloc[i] = df['c'].iloc[i] - df['c'].iloc[i - 10] 
        df['prev_24'].iloc[i] = df['c'].iloc[i] - df['c'].iloc[i - 24]    
        
        df['next_candle'].iloc[i] = df['c'].iloc[i + 1] - df['c'].iloc[i]
        df['next_2'].iloc[i] = df['c'].iloc[i + 2] - df['c'].iloc[i]
        df['next_3'].iloc[i] = df['c'].iloc[i + 3] - df['c'].iloc[i]
        df['next_4'].iloc[i] = df['c'].iloc[i + 4] - df['c'].iloc[i]
        df['next_5'].iloc[i] = df['c'].iloc[i + 5] - df['c'].iloc[i]
        df['next_10'].iloc[i] = df['c'].iloc[i + 10] - df['c'].iloc[i]
        
        df['next_candle+1'].iloc[i] = df['c'].iloc[i + 1+1] - df['c'].iloc[i+1]
        df['next_2+1'].iloc[i] = df['c'].iloc[i + 2+1] - df['c'].iloc[i+1]
        df['next_3+1'].iloc[i] = df['c'].iloc[i + 3+1] - df['c'].iloc[i+1]
        df['next_4+1'].iloc[i] = df['c'].iloc[i + 4+1] - df['c'].iloc[i+1]
        df['next_5+1'].iloc[i] = df['c'].iloc[i + 5+1] - df['c'].iloc[i+1]
        df['next_10+1'].iloc[i] = df['c'].iloc[i + 10+1] - df['c'].iloc[i+1]
        df['next_20+1'].iloc[i] = df['c'].iloc[i + 20+1] - df['c'].iloc[i+1]   
        
    return df
def test_method(df,positive = True,test_col = 'delta_percentile_prev_wick',percentile_test = 97):
    if positive:
        negative = False
    else:
        negative = True
    x = list(df.delta)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile'] = delta_percentile


    x = list(df.prev_delta)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile_prev'] = delta_percentile


    x = list(df.prev_delta_vector)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile_prev_vector'] = delta_percentile

    
    x = list(df.prev_wick)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile_prev_wick'] = delta_percentile
    
    
    x = list(df['prev_24'])
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_prev_24'] = delta_percentile    
    
    print(df.shape)

    if positive:
        top_x = df[(df[test_col] >= percentile_test) & (df['delta_vector'] < 0)]
    if negative:
        top_x = df[(df[test_col] < percentile_test)& (df['delta_vector'] > 0)]
        
    print(top_x.shape)    
    
    pos = []
    neg = []
    for i in range(top_x.shape[0]):
        val = round(top_x.delta_vector.iloc[i],4)
        if val < 0:
            neg.append(val)
        else:
            pos.append(val)

    print('POSITIVE:',len(pos),sum(pos),np.mean(pos),np.median(pos))
    print('NEGATIVE:',len(neg),sum(neg),np.mean(neg),np.median(neg))  
    print('OVERALL:',sum(top_x.delta_vector),np.mean(top_x.delta_vector),np.median(top_x.delta_vector))
    print('NEXT CANDLE FUTURE OVERALL:',sum(top_x.next_candle),np.mean(top_x.next_candle),
              np.median(top_x.next_candle),'count neg:',top_x[top_x.next_candle < 0].shape[0],'count pos:',top_x[top_x.next_candle > 0].shape[0])
    
    col = 'next_2'
    print('2 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0])    
    col = 'next_3'
    print('3 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0])    
    
    col = 'next_4'
    print('4 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0])    
    col = 'next_5'
    print('5 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0]) 
    col = 'next_10'
    print('10 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0]) 
    
    
def test_method(df,positive = True,test_col = 'delta_percentile_prev_wick',percentile_test = 97):
    print('TEST 2')
    if positive:
        negative = False
    else:
        negative = True
    x = list(df.delta)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile'] = delta_percentile


    x = list(df.prev_delta)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile_prev'] = delta_percentile


    x = list(df.prev_delta_vector)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile_prev_vector'] = delta_percentile

    
    x = list(df.prev_wick)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile_prev_wick'] = delta_percentile

    x = list(df['prev_24'])
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_prev_24'] = delta_percentile        
    
    print(df.shape)

    if positive:
        top_x = df[(df[test_col] >= percentile_test) & (df['delta_prev_24'] >= .8) & (df['delta_vector'] < 0)]
    if negative:
        top_x = df[(df[test_col] < percentile_test)& (df['delta_prev_24'] < .2)& (df['delta_vector'] > 0)]
        
    print(top_x.shape)    
    
    pos = []
    neg = []
    for i in range(top_x.shape[0]):
        val = round(top_x.delta_vector.iloc[i],4)
        if val < 0:
            neg.append(val)
        else:
            pos.append(val)

    print('POSITIVE:',len(pos),sum(pos),np.mean(pos),np.median(pos))
    print('NEGATIVE:',len(neg),sum(neg),np.mean(neg),np.median(neg))  
    print('OVERALL:',sum(top_x.delta_vector),np.mean(top_x.delta_vector),np.median(top_x.delta_vector))
    print('NEXT CANDLE FUTURE OVERALL:',sum(top_x.next_candle),np.mean(top_x.next_candle),
              np.median(top_x.next_candle),'count neg:',top_x[top_x.next_candle < 0].shape[0],'count pos:',top_x[top_x.next_candle > 0].shape[0])
    
    col = 'next_2'
    print('2 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0])    
    col = 'next_3'
    print('3 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0])    
    
    col = 'next_4'
    print('4 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0])    
    col = 'next_5'
    print('5 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0]) 
    col = 'next_10'
    print('10 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0]) 
    return top_x
    
    
def test_method(df,positive = True,test_col = 'delta_percentile_prev_wick',percentile_test = 97):
    print('TEST 2')
    if positive:
        negative = False
    else:
        negative = True
    x = list(df.delta)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile'] = delta_percentile


    x = list(df.prev_delta)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile_prev'] = delta_percentile


    x = list(df.prev_delta_vector)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile_prev_vector'] = delta_percentile

    
    x = list(df.prev_wick)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile_prev_wick'] = delta_percentile

    x = list(df['prev_24'])
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_prev_24'] = delta_percentile        
    
    print(df.shape)

    if positive:
        top_x = df[(df[test_col] >= percentile_test)]
    if negative:
        top_x = df[(df[test_col] < percentile_test)]
        
    print(top_x.shape)    
    
    pos = []
    neg = []
    for i in range(top_x.shape[0]):
        val = round(top_x.delta_vector.iloc[i],4)
        if val < 0:
            neg.append(val)
        else:
            pos.append(val)

    print('POSITIVE:',len(pos),sum(pos),np.mean(pos),np.median(pos))
    print('NEGATIVE:',len(neg),sum(neg),np.mean(neg),np.median(neg))  
    print('OVERALL:',sum(top_x.delta_vector),np.mean(top_x.delta_vector),np.median(top_x.delta_vector))
    print('NEXT CANDLE FUTURE OVERALL:',sum(top_x.next_candle),np.mean(top_x.next_candle),
              np.median(top_x.next_candle),'count neg:',top_x[top_x.next_candle < 0].shape[0],'count pos:',top_x[top_x.next_candle > 0].shape[0])
    
    col = 'next_2'
    print('2 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0])    
    col = 'next_3'
    print('3 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0])    
    
    col = 'next_4'
    print('4 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0])    
    col = 'next_5'
    print('5 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0]) 
    col = 'next_10'
    print('10 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0]) 
    return top_x
    
    
    
lst = all_pairs
lst = jpy_pairs
lst = [jpy_pairs[0]]
lst = ['XAU_USD']
dct = {}
for i in lst:    
    df = convert_to_df(instrument = i,granularity = "H1",count = 5000,percentile_test = 97,test_col = 'delta_percentile_prev_wick',positive = True)
    #top_x = test_method(df,positive = True,test_col = 'delta_percentile_prev_vector',percentile_test = 99)
    #top_x = test_method(df,positive = True,test_col = 'delta_percentile_prev_wick',percentile_test = 99.6)
    top_x = test_method(df,positive = True,test_col = 'delta_percentile_prev_vector',percentile_test = 99.5)
    
    dct[i] = top_x
    

df['trigger'] = 0

df.loc[df[df.time.isin(top_x.time)].index,'trigger'] = 1
print(df[df.trigger == 1].index[0])

df.head()



In [ ]:
np.percentile(df[col], 99.9)

In [ ]:
filter_df

In [ ]:
df.iloc[i-2:i+2,:14]

In [ ]:
col = 'prev_delta'
filter_df = df[df[col] >= np.percentile(df[col], 99.9)]
print('SHAPE',filter_df.shape)
filter_ind = filter_df.index


df['trigger'] = 0

df.loc[df[df.time.isin(filter_df.time)].index,'trigger'] = 1
print(df[df.trigger == 1].index[0])
i = df[df.trigger == 1].index[11]


fig = go.Figure(data=go.Candlestick(x=df.iloc[i-20:i + 50,:]['time'],
                    open=df.iloc[i-20:i + 50,:]['o'],
                    high=df.iloc[i-20:i + 50,:]['h'],
                    low=df.iloc[i-20:i + 50,:]['l'],
                    close=df.iloc[i-20:i + 50,:]['c']))


fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i], 
              annotation_text="trigger point " + str(round(df[col].iloc[i],4)), annotation_position="top left",
              fillcolor="green", opacity=0.25, line_width=0)


fig.show()


In [ ]:
df.shape[0]/24

In [ ]:


i = df[df.trigger == 1].index[7]


fig = go.Figure(data=go.Candlestick(x=df.iloc[i-20:i + 50,:]['time'],
                    open=df.iloc[i-20:i + 50,:]['o'],
                    high=df.iloc[i-20:i + 50,:]['h'],
                    low=df.iloc[i-20:i + 50,:]['l'],
                    close=df.iloc[i-20:i + 50,:]['c']))


fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i], 
              annotation_text="trigger point " + str(round(df['prev_wick'].iloc[i],4)), annotation_position="top left",
              fillcolor="green", opacity=0.25, line_width=0)


fig.show()


In [ ]:
i = 65
import plotly

fig = go.Figure(data=go.Ohlc(x=df.iloc[i-2:150,:]['time'],
                    open=df.iloc[i-2:150,:]['o'],
                    high=df.iloc[i-2:150,:]['h'],
                    low=df.iloc[i-2:150,:]['l'],
                    close=df.iloc[i-2:150,:]['c']))

fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
              annotation_text="trigger point", annotation_position="top left",
              fillcolor="green", opacity=0.25, line_width=0)
fig.show()

In [ ]:
from plotly.subplots import make_subplots
fig = make_subplots(rows=1, cols=2)
for i in range(2):
    
    print(i)
    
    
    i = df[df.trigger == 1].index[i]

    fig = go.Figure(data=go.Candlestick(x=df.iloc[i-2:150,:]['time'],
                        open=df.iloc[i-2:150,:]['o'],
                        high=df.iloc[i-2:150,:]['h'],
                        low=df.iloc[i-2:150,:]['l'],
                        close=df.iloc[i-2:150,:]['c']))

    fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i], 
                  annotation_text="trigger point " + str(round(df['delta'].iloc[i-1],4)), annotation_position="top left",
                  fillcolor="green", opacity=0.25, line_width=0)
    print(fig.show())

In [ ]:
top_x.shape

In [ ]:
import plotly

i = df[df.trigger == 1].index[6]

fig = go.Figure(data=go.Candlestick(x=df.iloc[i-20:i + 50,:]['time'],
                    open=df.iloc[i-20:i + 50,:]['o'],
                    high=df.iloc[i-20:i + 50,:]['h'],
                    low=df.iloc[i-20:i + 50,:]['l'],
                    close=df.iloc[i-20:i + 50,:]['c']))

fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i], 
              annotation_text="trigger point " + str(round(df['delta'].iloc[i-1],4)), annotation_position="top left",
              fillcolor="green", opacity=0.25, line_width=0)
fig.show()



In [ ]:
for i in range(10):
    print(i)
    i = df[df.trigger == 1].index[i]

    fig = go.Figure(data=go.Candlestick(x=df.iloc[i-2:150,:]['time'],
                        open=df.iloc[i-2:150,:]['o'],
                        high=df.iloc[i-2:150,:]['h'],
                        low=df.iloc[i-2:150,:]['l'],
                        close=df.iloc[i-2:150,:]['c']))

    fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i], 
                  annotation_text="trigger point " + str(round(df['delta'].iloc[i-1],4)), annotation_position="top left",
                  fillcolor="green", opacity=0.25, line_width=0)
    print(fig.show())

In [ ]:
df.iloc[i-2:i+1,:]

In [ ]:
df.head()

In [ ]:
df2 = df.copy()

In [ ]:
!pip install plotly

In [ ]:
import plotly.graph_objects as go

a = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')
a.head()


In [ ]:
a.shape

In [ ]:
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "iframe"
a = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')
a.head()

fig = go.Figure(data=go.Ohlc(x=a['Date'],
                    open=a['AAPL.Open'],
                    high=a['AAPL.High'],
                    low=a['AAPL.Low'],
                    close=a['AAPL.Close']))
fig.show()

In [ ]:
df.head()

In [ ]:
go

In [ ]:
df.columns

In [ ]:
df.columns

In [ ]:
import oandapyV20.endpoints.instruments as instruments

def test_wick3(instrument,granularity = "H1",count = 5000,percentile_test = 97,test_col = 'delta_percentile_prev_wick',positive = True):
    print()
    print()
    print()
    print(instrument,granularity,count,percentile_test)
    if positive:
        negative = False
    else:
        negative = True
    params = {"count":count,
              "granularity": granularity
             }
    print(instrument,type(instrument))
    r = instruments.InstrumentsCandles(instrument=instrument,
                                      params=params)
    client.request(r)
    dct = r.response
    lst = []
    i = 0
    
    for candle in dct['candles']:
        #IF CANDLE IS BULLISH
        if (float(candle['mid']['c']) - float(candle['mid']['o'])) > 0:
            max_vector = (float(candle['mid']['h']) - float(candle['mid']['l']))
            
        #IF CANDLE IS BEARISH
        else:
            max_vector = -1 * (float(candle['mid']['h']) - float(candle['mid']['l']))
            
        if i == 0:
            prev_delta = 0 
            prev_delta_max = 0
            prev_delta_vector = 0 
            prev_wick = 0
        else:
            prev_delta = abs(float(prev_candle['mid']['c']) - float(prev_candle['mid']['o']))
            prev_delta_max = abs(float(prev_candle['mid']['l']) - float(prev_candle['mid']['h']))
            prev_delta_vector = (float(prev_candle['mid']['c']) - float(prev_candle['mid']['o']))
            
            if (float(prev_candle['mid']['c']) - float(prev_candle['mid']['o'])) > 0:
                prev_wick = float(prev_candle['mid']['h']) - float(prev_candle['mid']['c'])
            else:
                prev_wick = float(prev_candle['mid']['l']) - float(prev_candle['mid']['o'])


        lst.append([candle['time'],candle['volume'],float(candle['mid']['o']),float(candle['mid']['h']),float(candle['mid']['l']),
                        float(candle['mid']['c']),abs(float(candle['mid']['c']) - float(candle['mid']['o'])),
                    abs(float(candle['mid']['l']) - float(candle['mid']['h'])),
                     (float(candle['mid']['c']) - float(candle['mid']['o'])),max_vector,

                    prev_delta,prev_delta_max,prev_delta_vector,prev_wick

                   ])
        
        prev_candle = candle
        i += 1


    df = pd.DataFrame(lst,columns = ['time','volume','o','h','l','c','delta','delta_max',
                                     'delta_vector','max_vector','prev_delta',
                                     'prev_delta_max','prev_delta_vector','prev_wick'])
    
    df['next_candle'] = 0
    df['next_3'] = 0
    df['next_5'] = 0
    df['next_10'] = 0
    df['next_20'] = 0
    
    tot_length = df.shape[0]
    for i in range(df.shape[0] - 20):
        df['next_candle'].iloc[i] = df['c'].iloc[i + 1] - df['c'].iloc[i]
        df['next_3'].iloc[i] = df['c'].iloc[i + 3] - df['c'].iloc[i]
        df['next_5'].iloc[i] = df['c'].iloc[i + 5] - df['c'].iloc[i]
        df['next_10'].iloc[i] = df['c'].iloc[i + 10] - df['c'].iloc[i]
        df['next_20'].iloc[i] = df['c'].iloc[i + 20] - df['c'].iloc[i]

        
    x = list(df.delta)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile'] = delta_percentile


    x = list(df.prev_delta)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile_prev'] = delta_percentile


    x = list(df.prev_delta_vector)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile_prev_vector'] = delta_percentile

    
    x = list(df.prev_wick)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile_prev_wick'] = delta_percentile
    
    print(df.shape)

    if positive:
        top_x = df[df[test_col] >= percentile_test]
    if negative:
        top_x = df[df[test_col] < percentile_test]
        
    print(top_x.shape)    
    
    pos = []
    neg = []
    for i in range(top_x.shape[0]):
        val = round(top_x.delta_vector.iloc[i],4)
        if val < 0:
            neg.append(val)
        else:
            pos.append(val)

    print('POSITIVE:',len(pos),sum(pos),np.mean(pos),np.median(pos))
    print('NEGATIVE:',len(neg),sum(neg),np.mean(neg),np.median(neg))  
    print('OVERALL:',sum(top_x.delta_vector),np.mean(top_x.delta_vector),np.median(top_x.delta_vector))
    print('NEXT CANDLE FUTURE OVERALL:',sum(top_x.next_candle),np.mean(top_x.next_candle),
              np.median(top_x.next_candle),'count neg:',top_x[top_x.next_candle < 0].shape[0],'count pos:',top_x[top_x.next_candle > 0].shape[0])
    col = 'next_3'
    print('3 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0])    
    col = 'next_5'
    print('5 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0]) 
    col = 'next_10'
    print('10 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0]) 
    col = 'next_20'
    print('20 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0]) 
    
def test_wick2(instrument,granularity = "H1",count = 5000,percentile_test = 97,test_col = 'delta_percentile_prev_wick',positive = True):
    print()
    print()
    print()
    print(instrument,granularity,count,percentile_test)
    if positive:
        negative = False
    else:
        negative = True
    params = {"count":count,
              "granularity": granularity
             }
    print(instrument,type(instrument))
    r = instruments.InstrumentsCandles(instrument=instrument,
                                      params=params)
    client.request(r)
    dct = r.response
    lst = []
    i = 0
    
    for candle in dct['candles']:
        if (float(candle['mid']['c']) - float(candle['mid']['o'])) > 0:
            max_vector = (float(candle['mid']['h']) - float(candle['mid']['l']))
        else:
            max_vector = -1 * (float(candle['mid']['h']) - float(candle['mid']['l']))
            
        if i == 0:
            prev_delta = 0 
            prev_delta_max = 0
            prev_delta_vector = 0 
            prev_wick = 0
        else:
            prev_delta = abs(float(prev_candle['mid']['c']) - float(prev_candle['mid']['o']))
            prev_delta_max = abs(float(prev_candle['mid']['l']) - float(prev_candle['mid']['h']))
            prev_delta_vector = (float(prev_candle['mid']['c']) - float(prev_candle['mid']['o']))
            
            if (float(prev_candle['mid']['c']) - float(prev_candle['mid']['o'])) > 0:
                prev_wick = float(prev_candle['mid']['h']) - float(prev_candle['mid']['c'])
            else:
                prev_wick = float(prev_candle['mid']['l']) - float(prev_candle['mid']['o'])


        lst.append([candle['time'],candle['volume'],float(candle['mid']['o']),float(candle['mid']['h']),float(candle['mid']['l']),
                        float(candle['mid']['c']),abs(float(candle['mid']['c']) - float(candle['mid']['o'])),
                    abs(float(candle['mid']['l']) - float(candle['mid']['h'])),
                     (float(candle['mid']['c']) - float(candle['mid']['o'])),max_vector,

                    prev_delta,prev_delta_max,prev_delta_vector,prev_wick

                   ])
        
        prev_candle = candle
        i += 1


    df = pd.DataFrame(lst,columns = ['time','volume','o','h','l','c','delta','delta_max',
                                     'delta_vector','max_vector','prev_delta',
                                     'prev_delta_max','prev_delta_vector','prev_wick'])
    
    df['next_candle'] = 0
    df['next_3'] = 0
    df['next_5'] = 0
    df['next_10'] = 0
    df['next_20'] = 0
    
    tot_length = df.shape[0]
    for i in range(df.shape[0] - 20):
        df['next_candle'].iloc[i] = df['c'].iloc[i + 1] - df['c'].iloc[i]
        df['next_3'].iloc[i] = df['c'].iloc[i + 3] - df['c'].iloc[i]
        df['next_5'].iloc[i] = df['c'].iloc[i + 5] - df['c'].iloc[i]
        df['next_10'].iloc[i] = df['c'].iloc[i + 10] - df['c'].iloc[i]
        df['next_20'].iloc[i] = df['c'].iloc[i + 20] - df['c'].iloc[i]

        
    x = list(df.delta)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile'] = delta_percentile


    x = list(df.prev_delta)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile_prev'] = delta_percentile


    x = list(df.prev_delta_vector)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile_prev_vector'] = delta_percentile

    
    x = list(df.prev_wick)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile_prev_wick'] = delta_percentile
    
    print(df.shape)

    if positive:
        top_x = df[df[test_col] >= percentile_test]
    if negative:
        top_x = df[df[test_col] < percentile_test]
    print(top_x.shape)    
    
    pos = []
    neg = []
    for i in range(top_x.shape[0]):
        val = round(top_x.delta_vector.iloc[i],4)
        if val < 0:
            neg.append(val)
        else:
            pos.append(val)

    print('POSITIVE:',len(pos),sum(pos),np.mean(pos),np.median(pos))
    print('NEGATIVE:',len(neg),sum(neg),np.mean(neg),np.median(neg))  
    print('OVERALL:',sum(top_x.delta_vector),np.mean(top_x.delta_vector),np.median(top_x.delta_vector))
    print('NEXT CANDLE FUTURE OVERALL:',sum(top_x.next_candle),np.mean(top_x.next_candle),
              np.median(top_x.next_candle),'count neg:',top_x[top_x.next_candle < 0].shape[0],'count pos:',top_x[top_x.next_candle > 0].shape[0])
    col = 'next_3'
    print('3 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0])    
    col = 'next_5'
    print('5 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0]) 
    col = 'next_10'
    print('10 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0]) 
    col = 'next_20'
    print('20 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0]) 
    
    
def test_wick(instrument,granularity = "H1",count = 5000,percentile_test = 97,test_col = 'delta_percentile_prev_wick',positive = True):
    print()
    print()
    print()
    print(instrument,granularity,count,percentile_test)
    if positive:
        negative = False
    else:
        negative = True
    params = {"count":count,
              "granularity": granularity
             }
    print(instrument,type(instrument))
    r = instruments.InstrumentsCandles(instrument=instrument,
                                      params=params)
    client.request(r)
    dct = r.response
    lst = []
    i = 0
    for candle in dct['candles']:
        if (float(candle['mid']['c']) - float(candle['mid']['o'])) > 0:
            max_vector = (float(candle['mid']['h']) - float(candle['mid']['l']))
        else:
            max_vector = -1 * (float(candle['mid']['h']) - float(candle['mid']['l']))
        if i == 0:
            prev_delta = 0 
            prev_delta_max = 0
            prev_delta_vector = 0 
            prev_wick = 0
        else:
            prev_delta = abs(float(prev_candle['mid']['c']) - float(prev_candle['mid']['o']))
            prev_delta_max = abs(float(prev_candle['mid']['l']) - float(prev_candle['mid']['h']))
            prev_delta_vector = (float(prev_candle['mid']['c']) - float(prev_candle['mid']['o']))
            if (float(prev_candle['mid']['c']) - float(prev_candle['mid']['o'])) > 0:
                prev_wick = float(prev_candle['mid']['h']) - float(prev_candle['mid']['c'])
            else:
                prev_wick = float(prev_candle['mid']['l']) - float(prev_candle['mid']['o'])


        lst.append([candle['time'],candle['volume'],float(candle['mid']['o']),float(candle['mid']['h']),float(candle['mid']['l']),
                        float(candle['mid']['c']),abs(float(candle['mid']['c']) - float(candle['mid']['o'])),
                    abs(float(candle['mid']['l']) - float(candle['mid']['h'])),
                     (float(candle['mid']['c']) - float(candle['mid']['o'])),max_vector,

                    prev_delta,prev_delta_max,prev_delta_vector,prev_wick

                   ])
        prev_candle = candle
        i += 1


    df = pd.DataFrame(lst,columns = ['time','volume','o','h','l','c','delta','delta_max',
                                     'delta_vector','max_vector','prev_delta',
                                     'prev_delta_max','prev_delta_vector','prev_wick'])
    df['next_candle'] = 0
    df['next_3'] = 0
    df['next_5'] = 0
    df['next_10'] = 0
    df['next_20'] = 0
    
    tot_length = df.shape[0]
    for i in range(df.shape[0] - 20):
        df['next_candle'].iloc[i] = df['c'].iloc[i + 1] - df['c'].iloc[i]
        df['next_3'].iloc[i] = df['c'].iloc[i + 3] - df['c'].iloc[i]
        df['next_5'].iloc[i] = df['c'].iloc[i + 5] - df['c'].iloc[i]
        df['next_10'].iloc[i] = df['c'].iloc[i + 10] - df['c'].iloc[i]
        df['next_20'].iloc[i] = df['c'].iloc[i + 20] - df['c'].iloc[i]

        
    x = list(df.delta)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile'] = delta_percentile


    x = list(df.prev_delta)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile_prev'] = delta_percentile


    x = list(df.prev_delta_vector)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile_prev_vector'] = delta_percentile

    
    x = list(df.prev_wick)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile_prev_wick'] = delta_percentile
    
    print(df.shape)

    if positive:
        top_x = df[df[test_col] >= percentile_test]
    if negative:
        top_x = df[df[test_col] < percentile_test]
    print(top_x.shape)    
    
    pos = []
    neg = []
    for i in range(top_x.shape[0]):
        val = round(top_x.delta_vector.iloc[i],4)
        if val < 0:
            neg.append(val)
        else:
            pos.append(val)

    print('POSITIVE:',len(pos),sum(pos),np.mean(pos),np.median(pos))
    print('NEGATIVE:',len(neg),sum(neg),np.mean(neg),np.median(neg))  
    print('OVERALL:',sum(top_x.delta_vector),np.mean(top_x.delta_vector),np.median(top_x.delta_vector))
    print('NEXT CANDLE FUTURE OVERALL:',sum(top_x.next_candle),np.mean(top_x.next_candle),
              np.median(top_x.next_candle),'count neg:',top_x[top_x.next_candle < 0].shape[0],'count pos:',top_x[top_x.next_candle > 0].shape[0])
    col = 'next_3'
    print('3 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0])    
    col = 'next_5'
    print('5 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0]) 
    col = 'next_10'
    print('10 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0]) 
    col = 'next_20'
    print('20 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0]) 
    
def test(instrument="GBP_JPY",granularity = "H1",count = 5000,percentile_test = 97):

    params = {"count":count,
              "granularity": granularity
             }
    r = instruments.InstrumentsCandles(instrument=instrument,
                                      params=params)
    client.request(r)
    dct = r.response
    lst = []
    i = 0
    for candle in dct['candles']:
        if (float(candle['mid']['c']) - float(candle['mid']['o'])) > 0:
            max_vector = (float(candle['mid']['h']) - float(candle['mid']['l']))
        else:
            max_vector = -1 * (float(candle['mid']['h']) - float(candle['mid']['l']))
        if i == 0:
            prev_delta = 0 
            prev_delta_max = 0
            prev_delta_vector = 0 
        else:
            prev_delta = abs(float(prev_candle['mid']['c']) - float(prev_candle['mid']['o']))
            prev_delta_max = abs(float(prev_candle['mid']['l']) - float(prev_candle['mid']['h']))
            prev_delta_vector = (float(prev_candle['mid']['c']) - float(prev_candle['mid']['o']))


        lst.append([candle['time'],candle['volume'],float(candle['mid']['o']),float(candle['mid']['h']),float(candle['mid']['l']),
                        float(candle['mid']['c']),abs(float(candle['mid']['c']) - float(candle['mid']['o'])),
                    abs(float(candle['mid']['l']) - float(candle['mid']['h'])),
                     (float(candle['mid']['c']) - float(candle['mid']['o'])),max_vector,

                    prev_delta,prev_delta_max,prev_delta_vector

                   ])
        prev_candle = candle
        i += 1


    df = pd.DataFrame(lst,columns = ['time','volume','o','h','l','c','delta','delta_max',
                                     'delta_vector','max_vector','prev_delta',
                                     'prev_delta_max','prev_delta_vector'])
    x = list(df.delta)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile'] = delta_percentile


    x = list(df.prev_delta)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile_prev'] = delta_percentile


    x = list(df.prev_delta_vector)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile_prev_vector'] = delta_percentile

    print(df.shape)
    if positive:
        top_x = df[df.delta_percentile_prev_vector >= percentile_test]
    if negative:
        top_x = df[df.delta_percentile_prev_vector <= percentile_test]
    print(top_x.shape)    
    
    pos = []
    neg = []
    for i in range(top_x.shape[0]):
        val = round(top_x.delta_vector.iloc[i],4)
        if val < 0:
            neg.append(val)
        else:
            pos.append(val)

    print('POSITIVE:',len(pos),sum(pos),np.mean(pos),np.median(pos))
    print('NEGATIVE:',len(neg),sum(neg),np.mean(neg),np.median(neg))  
    print('OVERALL:',sum(top_x.delta_vector),np.mean(top_x.delta_vector),np.median(top_x.delta_vector))

#test(instrument="GBP_JPY",granularity = "H1",count = 5000,percentile_test = 97)   
lst = all_pairs
for i in lst:
    #'delta_percentile_prev_vector'
    #test_wick(instrument=i,granularity = "H1",count = 5000,percentile_test = 98,test_col = 'delta_percentile_prev_wick') 
    test_wick2(instrument,granularity = "H1",count = 5000,percentile_test = 97,test_col = 'delta_percentile_prev_wick',positive = True)

In [ ]:
lst = all_pairs
for i in lst:
    #'delta_percentile_prev_vector'
    #test_wick(instrument=i,granularity = "H1",count = 5000,percentile_test = 98,test_col = 'delta_percentile_prev_wick') 
    test_wick2(instrument = i,granularity = "H1",count = 5000,percentile_test = 98,test_col = 'delta_percentile_prev_wick',positive = True)
    
    

In [ ]:
import oandapyV20
import oandapyV20.endpoints.instruments as instruments

params = {"count":5,
          "granularity": "H1"
         }
r = instruments.InstrumentsCandles(instrument="GBP_JPY",
                                  params=params)
client.request(r)
print (r.response)

In [ ]:
params = {"count":5000,
          "granularity": "H1"
         }
r = instruments.InstrumentsCandles(instrument="GBP_JPY",
                                  params=params)
client.request(r)
dct = r.response
lst = []
i = 0
for candle in dct['candles']:
    if (float(candle['mid']['c']) - float(candle['mid']['o'])) > 0:
        max_vector = (float(candle['mid']['h']) - float(candle['mid']['l']))
    else:
        max_vector = -1 * (float(candle['mid']['h']) - float(candle['mid']['l']))
    if i == 0:
        prev_delta = 0 
        prev_delta_max = 0
        prev_delta_vector = 0 
    else:
        prev_delta = abs(float(prev_candle['mid']['c']) - float(prev_candle['mid']['o']))
        prev_delta_max = abs(float(prev_candle['mid']['l']) - float(prev_candle['mid']['h']))
        prev_delta_vector = (float(prev_candle['mid']['c']) - float(prev_candle['mid']['o']))
        
        
    lst.append([candle['time'],candle['volume'],float(candle['mid']['o']),float(candle['mid']['h']),float(candle['mid']['l']),
                    float(candle['mid']['c']),abs(float(candle['mid']['c']) - float(candle['mid']['o'])),abs(float(candle['mid']['l']) - float(candle['mid']['h'])),
                 (float(candle['mid']['c']) - float(candle['mid']['o'])),max_vector,
                
                prev_delta,prev_delta_max,prev_delta_vector
               
               ])
    prev_candle = candle
    i += 1
    
    
df = pd.DataFrame(lst,columns = ['time','volume','o','h','l','c','delta','delta_max','delta_vector','max_vector','prev_delta','prev_delta_max','prev_delta_vector'])
x = list(df.delta)
delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
df['delta_percentile'] = delta_percentile


x = list(df.prev_delta)
delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
df['delta_percentile_prev'] = delta_percentile


x = list(df.prev_delta_vector)
delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
df['delta_percentile_prev_vector'] = delta_percentile
 
print(df.shape)
df.head()

In [ ]:
pos = []
neg = []
for i in range(top_95.shape[0]):
    val = round(top_95.delta_vector.iloc[i],4)
    if val < 0:
        neg.append(val)
    else:
        pos.append(val)
    #print(round(top_95.delta_vector.iloc[i],4))
print(sum(pos),sum(neg))

In [ ]:
import oandapyV20.endpoints.instruments as instruments
def test_wick2(instrument,granularity = "H1",count = 5000,percentile_test = 97,test_col = 'delta_percentile_prev_wick',positive = True):
    print()
    print()
    print()
    print(instrument,granularity,count,percentile_test)
    if positive:
        negative = False
    else:
        negative = True
    params = {"count":count,
              "granularity": granularity
             }
    print(instrument,type(instrument))
    r = instruments.InstrumentsCandles(instrument=instrument,
                                      params=params)
    client.request(r)
    dct = r.response
    lst = []
    i = 0
    for candle in dct['candles']:
        if (float(candle['mid']['c']) - float(candle['mid']['o'])) > 0:
            max_vector = (float(candle['mid']['h']) - float(candle['mid']['l']))
        else:
            max_vector = -1 * (float(candle['mid']['h']) - float(candle['mid']['l']))
        if i == 0:
            prev_delta = 0 
            prev_delta_max = 0
            prev_delta_vector = 0 
            prev_wick = 0
        else:
            prev_delta = abs(float(prev_candle['mid']['c']) - float(prev_candle['mid']['o']))
            prev_delta_max = abs(float(prev_candle['mid']['l']) - float(prev_candle['mid']['h']))
            prev_delta_vector = (float(prev_candle['mid']['c']) - float(prev_candle['mid']['o']))
            if (float(prev_candle['mid']['c']) - float(prev_candle['mid']['o'])) > 0:
                prev_wick = float(prev_candle['mid']['h']) - float(prev_candle['mid']['c'])
            else:
                prev_wick = float(prev_candle['mid']['l']) - float(prev_candle['mid']['o'])


        lst.append([candle['time'],candle['volume'],float(candle['mid']['o']),float(candle['mid']['h']),float(candle['mid']['l']),
                        float(candle['mid']['c']),abs(float(candle['mid']['c']) - float(candle['mid']['o'])),
                    abs(float(candle['mid']['l']) - float(candle['mid']['h'])),
                     (float(candle['mid']['c']) - float(candle['mid']['o'])),max_vector,

                    prev_delta,prev_delta_max,prev_delta_vector,prev_wick

                   ])
        prev_candle = candle
        i += 1


    df = pd.DataFrame(lst,columns = ['time','volume','o','h','l','c','delta','delta_max',
                                     'delta_vector','max_vector','prev_delta',
                                     'prev_delta_max','prev_delta_vector','prev_wick'])
    df['next_candle'] = 0
    df['next_3'] = 0
    df['next_5'] = 0
    df['next_10'] = 0
    df['next_20'] = 0
    
    tot_length = df.shape[0]
    for i in range(df.shape[0] - 20):
        df['next_candle'].iloc[i] = df['c'].iloc[i + 1] - df['c'].iloc[i]
        df['next_3'].iloc[i] = df['c'].iloc[i + 3] - df['c'].iloc[i]
        df['next_5'].iloc[i] = df['c'].iloc[i + 5] - df['c'].iloc[i]
        df['next_10'].iloc[i] = df['c'].iloc[i + 10] - df['c'].iloc[i]
        df['next_20'].iloc[i] = df['c'].iloc[i + 20] - df['c'].iloc[i]

        
    x = list(df.delta)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile'] = delta_percentile


    x = list(df.prev_delta)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile_prev'] = delta_percentile


    x = list(df.prev_delta_vector)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile_prev_vector'] = delta_percentile

    
    x = list(df.prev_wick)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile_prev_wick'] = delta_percentile
    
    print(df.shape)

    if positive:
        top_x = df[df[test_col] >= percentile_test]
    if negative:
        top_x = df[df[test_col] < percentile_test]
    print(top_x.shape)    
    
    pos = []
    neg = []
    for i in range(top_x.shape[0]):
        val = round(top_x.delta_vector.iloc[i],4)
        if val < 0:
            neg.append(val)
        else:
            pos.append(val)

    print('POSITIVE:',len(pos),sum(pos),np.mean(pos),np.median(pos))
    print('NEGATIVE:',len(neg),sum(neg),np.mean(neg),np.median(neg))  
    print('OVERALL:',sum(top_x.delta_vector),np.mean(top_x.delta_vector),np.median(top_x.delta_vector))
    print('NEXT CANDLE FUTURE OVERALL:',sum(top_x.next_candle),np.mean(top_x.next_candle),
              np.median(top_x.next_candle),'count neg:',top_x[top_x.next_candle < 0].shape[0],'count pos:',top_x[top_x.next_candle > 0].shape[0])
    col = 'next_3'
    print('3 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0])    
    col = 'next_5'
    print('5 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0]) 
    col = 'next_10'
    print('10 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0]) 
    col = 'next_20'
    print('20 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0]) 
def test_wick(instrument,granularity = "H1",count = 5000,percentile_test = 97,test_col = 'delta_percentile_prev_wick',positive = True):
    print()
    print()
    print()
    print(instrument,granularity,count,percentile_test)
    if positive:
        negative = False
    else:
        negative = True
    params = {"count":count,
              "granularity": granularity
             }
    print(instrument,type(instrument))
    r = instruments.InstrumentsCandles(instrument=instrument,
                                      params=params)
    client.request(r)
    dct = r.response
    lst = []
    i = 0
    for candle in dct['candles']:
        if (float(candle['mid']['c']) - float(candle['mid']['o'])) > 0:
            max_vector = (float(candle['mid']['h']) - float(candle['mid']['l']))
        else:
            max_vector = -1 * (float(candle['mid']['h']) - float(candle['mid']['l']))
        if i == 0:
            prev_delta = 0 
            prev_delta_max = 0
            prev_delta_vector = 0 
            prev_wick = 0
        else:
            prev_delta = abs(float(prev_candle['mid']['c']) - float(prev_candle['mid']['o']))
            prev_delta_max = abs(float(prev_candle['mid']['l']) - float(prev_candle['mid']['h']))
            prev_delta_vector = (float(prev_candle['mid']['c']) - float(prev_candle['mid']['o']))
            if (float(prev_candle['mid']['c']) - float(prev_candle['mid']['o'])) > 0:
                prev_wick = float(prev_candle['mid']['h']) - float(prev_candle['mid']['c'])
            else:
                prev_wick = float(prev_candle['mid']['l']) - float(prev_candle['mid']['o'])


        lst.append([candle['time'],candle['volume'],float(candle['mid']['o']),float(candle['mid']['h']),float(candle['mid']['l']),
                        float(candle['mid']['c']),abs(float(candle['mid']['c']) - float(candle['mid']['o'])),
                    abs(float(candle['mid']['l']) - float(candle['mid']['h'])),
                     (float(candle['mid']['c']) - float(candle['mid']['o'])),max_vector,

                    prev_delta,prev_delta_max,prev_delta_vector,prev_wick

                   ])
        prev_candle = candle
        i += 1


    df = pd.DataFrame(lst,columns = ['time','volume','o','h','l','c','delta','delta_max',
                                     'delta_vector','max_vector','prev_delta',
                                     'prev_delta_max','prev_delta_vector','prev_wick'])
    df['next_candle'] = 0
    df['next_3'] = 0
    df['next_5'] = 0
    df['next_10'] = 0
    df['next_20'] = 0
    
    tot_length = df.shape[0]
    for i in range(df.shape[0] - 20):
        df['next_candle'].iloc[i] = df['c'].iloc[i + 1] - df['c'].iloc[i]
        df['next_3'].iloc[i] = df['c'].iloc[i + 3] - df['c'].iloc[i]
        df['next_5'].iloc[i] = df['c'].iloc[i + 5] - df['c'].iloc[i]
        df['next_10'].iloc[i] = df['c'].iloc[i + 10] - df['c'].iloc[i]
        df['next_20'].iloc[i] = df['c'].iloc[i + 20] - df['c'].iloc[i]

        
    x = list(df.delta)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile'] = delta_percentile


    x = list(df.prev_delta)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile_prev'] = delta_percentile


    x = list(df.prev_delta_vector)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile_prev_vector'] = delta_percentile

    
    x = list(df.prev_wick)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile_prev_wick'] = delta_percentile
    
    print(df.shape)

    if positive:
        top_x = df[df[test_col] >= percentile_test]
    if negative:
        top_x = df[df[test_col] < percentile_test]
    print(top_x.shape)    
    
    pos = []
    neg = []
    for i in range(top_x.shape[0]):
        val = round(top_x.delta_vector.iloc[i],4)
        if val < 0:
            neg.append(val)
        else:
            pos.append(val)

    print('POSITIVE:',len(pos),sum(pos),np.mean(pos),np.median(pos))
    print('NEGATIVE:',len(neg),sum(neg),np.mean(neg),np.median(neg))  
    print('OVERALL:',sum(top_x.delta_vector),np.mean(top_x.delta_vector),np.median(top_x.delta_vector))
    print('NEXT CANDLE FUTURE OVERALL:',sum(top_x.next_candle),np.mean(top_x.next_candle),
              np.median(top_x.next_candle),'count neg:',top_x[top_x.next_candle < 0].shape[0],'count pos:',top_x[top_x.next_candle > 0].shape[0])
    col = 'next_3'
    print('3 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0])    
    col = 'next_5'
    print('5 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0]) 
    col = 'next_10'
    print('10 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0]) 
    col = 'next_20'
    print('20 CANDLES FUTURE OVERALL:',sum(top_x[col]),np.mean(top_x[col]),
              np.median(top_x[col]),'count neg:',top_x[top_x[col] < 0].shape[0],'count pos:',top_x[top_x[col] > 0].shape[0]) 
    
def test(instrument="GBP_JPY",granularity = "H1",count = 5000,percentile_test = 97):

    params = {"count":count,
              "granularity": granularity
             }
    r = instruments.InstrumentsCandles(instrument=instrument,
                                      params=params)
    client.request(r)
    dct = r.response
    lst = []
    i = 0
    for candle in dct['candles']:
        if (float(candle['mid']['c']) - float(candle['mid']['o'])) > 0:
            max_vector = (float(candle['mid']['h']) - float(candle['mid']['l']))
        else:
            max_vector = -1 * (float(candle['mid']['h']) - float(candle['mid']['l']))
        if i == 0:
            prev_delta = 0 
            prev_delta_max = 0
            prev_delta_vector = 0 
        else:
            prev_delta = abs(float(prev_candle['mid']['c']) - float(prev_candle['mid']['o']))
            prev_delta_max = abs(float(prev_candle['mid']['l']) - float(prev_candle['mid']['h']))
            prev_delta_vector = (float(prev_candle['mid']['c']) - float(prev_candle['mid']['o']))


        lst.append([candle['time'],candle['volume'],float(candle['mid']['o']),float(candle['mid']['h']),float(candle['mid']['l']),
                        float(candle['mid']['c']),abs(float(candle['mid']['c']) - float(candle['mid']['o'])),
                    abs(float(candle['mid']['l']) - float(candle['mid']['h'])),
                     (float(candle['mid']['c']) - float(candle['mid']['o'])),max_vector,

                    prev_delta,prev_delta_max,prev_delta_vector

                   ])
        prev_candle = candle
        i += 1


    df = pd.DataFrame(lst,columns = ['time','volume','o','h','l','c','delta','delta_max',
                                     'delta_vector','max_vector','prev_delta',
                                     'prev_delta_max','prev_delta_vector'])
    x = list(df.delta)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile'] = delta_percentile


    x = list(df.prev_delta)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile_prev'] = delta_percentile


    x = list(df.prev_delta_vector)
    delta_percentile = [stats.percentileofscore(x, a, 'rank') for a in x]
    df['delta_percentile_prev_vector'] = delta_percentile

    print(df.shape)
    if positive:
        top_x = df[df.delta_percentile_prev_vector >= percentile_test]
    if negative:
        top_x = df[df.delta_percentile_prev_vector <= percentile_test]
    print(top_x.shape)    
    
    pos = []
    neg = []
    for i in range(top_x.shape[0]):
        val = round(top_x.delta_vector.iloc[i],4)
        if val < 0:
            neg.append(val)
        else:
            pos.append(val)

    print('POSITIVE:',len(pos),sum(pos),np.mean(pos),np.median(pos))
    print('NEGATIVE:',len(neg),sum(neg),np.mean(neg),np.median(neg))  
    print('OVERALL:',sum(top_x.delta_vector),np.mean(top_x.delta_vector),np.median(top_x.delta_vector))

#test(instrument="GBP_JPY",granularity = "H1",count = 5000,percentile_test = 97)   
lst = ["GBP_JPY",
              "USD_JPY",
               "EUR_USD",
               "EUR_JPY",
               "GBP_USD",
               "EUR_GBP"
              
              ]
for i in lst:
    #'delta_percentile_prev_vector'
    test_wick(instrument=i,granularity = "H1",count = 5000,percentile_test = 98,test_col = 'delta_percentile_prev_wick') 

In [ ]:
63 / (63 + 36)

In [ ]:
for i in lst:
    #'delta_percentile_prev_vector'
    test_wick(instrument=i,granularity = "H1",count = 5000,percentile_test = 98,test_col = 'delta_percentile_prev_vector',positive = True) 

In [ ]:
for i in lst:
    #'delta_percentile_prev_vector'
    test_wick(instrument=i,granularity = "H1",count = 5000,percentile_test = 2,test_col = 'delta_percentile_prev_vector',positive = False) 
    
    
    
    

In [ ]:
for i in lst:
    #'delta_percentile_prev_vector'
    test_wick(instrument=i,granularity = "H1",count = 5000,percentile_test = 1,test_col = 'delta_percentile_prev_vector',positive = False) 
    
    
    

In [ ]:
15000 / 24

In [ ]:
lst = ["GBP_JPY",
              "USD_JPY",
               "EUR_USD",
               "EUR_JPY",
               "GBP_USD",
               "EUR_GBP"
              
              ]
for i in lst:
    test_wick(instrument=i,granularity = "H1",count = 5000,percentile_test = 5,test_col = 'delta_percentile_prev_wick',positive = False) 
    
    

In [ ]:
test(instrument="USD_JPY",granularity = "H1",count = 5000,percentile_test = 97)   

In [ ]:
test(instrument="USD_JPY",granularity = "H1",count = 5000,percentile_test = 98)   

In [ ]:
test(instrument="USD_JPY",granularity = "H1",count = 5000,percentile_test = 98)   

In [ ]:
test(instrument="EUR_USD",granularity = "H1",count = 5000,percentile_test = 98)   

In [ ]:
test(instrument="EUR_JPY",granularity = "H1",count = 5000,percentile_test = 98)   

In [ ]:
test(instrument="GBP_USD",granularity = "H1",count = 5000,percentile_test = 98)   

In [ ]:
test(instrument="EUR_GBP",granularity = "H1",count = 5000,percentile_test = 90)   

In [ ]:
top_95 = df[df.delta_percentile_prev_vector >= 95]
print(top_95.shape)
top_95.head()

In [ ]:
top_95 = df[df.delta_percentile_prev >= 95]
print(top_95.shape)
top_95.head()

In [ ]:
np.mean(top_95.delta_percentile)

In [ ]:
top_95 = df[df.delta_percentile >= 95]
print(top_95.shape)
top_95.head()


In [ ]:
2500 / 24

In [ ]:
from scipy import stats
x = [1, 1, 2, 2, 17]
[stats.percentileofscore(x, a, 'rank') for a in x]

In [ ]:
x = list(df.delta)
new = [stats.percentileofscore(x, a, 'rank') for a in x]
new[:10]

In [ ]:
np.percentile(df.delta,80)

In [ ]:
df2 = df.sort_values(by = 'delta')
df2.tail()

In [ ]:
candle['mid']['c']

In [ ]:
24 * 30

In [ ]:
from oandapyV20 import API
import oandapyV20.endpoints.trades as trades



r = trades.TradesList(accountID)
# show the endpoint as it is constructed for this call
print("REQUEST:{}".format(r))
rv = api.request(r)
print("RESPONSE:\n{}".format(json.dumps(rv, indent=2)))

In [ ]:
import time
params ={
          "instruments": "EUR_USD"
        }
r = pricing.PricingInfo(accountID=accountID, params=params)
lst = []
steps = 10800
for i in range(steps):
    s = time.time()
    rv = api.request(r)
    rv['prices'][0]['time']

    lst.append([rv['time'],rv['prices'][0]['time'],rv['prices'][0]['closeoutBid'],rv['prices'][0]['closeoutAsk']])
   # print(rv['time'],'SECONDS:',time.time() - s)
    time.sleep(1)
df = pd.DataFrame(lst,columns = ['time','price_time','bid','ask'])



In [ ]:
df.head()

In [ ]:
60 * 60 * 3

In [ ]:
df.head()

In [ ]:
df.tail(50)

In [ ]:
pd.DataFrame(lst,columns = ['time','price_time','bid','ask'])

In [ ]:
rv

In [ ]:
rv['prices'][0]['time']
rv['prices'][0]['closeoutBid']
rv['prices'][0]['closeoutAsk']

In [ ]:

import time
s=time.time()

for i in range(100):
    rv = api.request(r)
e=time.time()
e-s
    

In [ ]:
price = float(rv['positionBook']['price'])

In [ ]:
{'positionBook': {'instrument': 'EUR_USD',
  'time': '2021-07-22T22:20:00Z',
  'unixTime': '1626992400',
  'price': '1.17750',
  'bucketWidth': '0.00050',
  'buckets': [{'price':

In [ ]:
bid = float(rv['prices'][0]['bids'][0]['price'])
ask = float(rv['prices'][0]['asks'][0]['price'])
print(bid,ask)
print(bid-.0005)

In [ ]:
delta = .005
print(price,price - delta,price + delta)
df[(df.price >= price - delta) & (df.price <= price + delta) ]

In [ ]:
delta = .005
print(price,price - delta,price + delta)
df2[(df2.price >= price - delta) & (df2.price <= price + delta) ]

In [ ]:
dct['positionBook']['time']

In [ ]:
params={'bucketWidth': '0.00010',
       'time': '2020-05-22T22:00:00Z'}
r = instruments.InstrumentsPositionBook(instrument="EUR_USD",
                                        params=params)
r2 = instruments.InstrumentsOrderBook(instrument="EUR_USD",
                                        params=params)

dct = client.request(r)
df = pd.DataFrame(dct['positionBook']['buckets'])
df = df.astype(float)
dct2 = client.request(r2)
df2 = pd.DataFrame(dct['positionBook']['buckets'])
df2 = df2.astype(float)
df2

In [ ]:
import oandapyV20
import oandapyV20.endpoints.forexlabs as labs
params ={
          "instrument": "EUR_USD",
          "period": 3600
        }
    
r = labs.HistoricalPositionRatios(params=params)
client.request(r)
print(r.response)

In [ ]:
api
params={'bucketWidth': '0.00010'}
r = instruments.InstrumentsPositionBook(instrument="EUR_USD",
                                        params=params)
r2 = instruments.InstrumentsOrderBook(instrument="EUR_USD",
                                        params=params)
params ={
          "instrument": "EUR_USD",
          "period": 60
        }

dct = client.request(r)
df = pd.DataFrame(dct['positionBook']['buckets'])
df = df.astype(float)
df[(df.price>(bid-.007)) & (df.price< (ask + .007))]


In [ ]:
df.head(50)

In [ ]:
params ={
          "instruments": "EUR_USD,EUR_JPY"
        }
r = pricing.PricingStream(accountID=accountID, params=params)
rv = api.request(r)
maxrecs = 100
for ticks in rv:
    print (json.dumps(ticks, indent=4),",")
    if maxrecs == 0:
        r.terminate("maxrecs records received")

In [ ]:
def past_data(pair,cnt,far_back):
    
    params = {
          "count": cnt,
          "granularity": "H1"}
    
    r = instruments.InstrumentsCandles(instrument=pair,
                                   params=params)
    a=api.request(r)
    
    
    Date_Time=[]
    O=[]
    H=[]
    L=[]
    C=[]
    Volume=[]
    C_minus_O=[]
    H_minus_L=[]
    for i in range(cnt):
        
        if (float(a['candles'][i]['volume']))!=0:
            Date_Time.append(a['candles'][i]['time'])
            O.append(float(a['candles'][i]['mid']['o']))
            H.append(float(a['candles'][i]['mid']['h']))
            L.append(float(a['candles'][i]['mid']['l']))
            C.append(float(a['candles'][i]['mid']['c']))
            Volume.append(float(a['candles'][i]['volume']))
            C_minus_O.append(float(a['candles'][i]['mid']['c'])-float(a['candles'][i]['mid']['o']))
            H_minus_L.append(float(a['candles'][i]['mid']['h'])-float(a['candles'][i]['mid']['l']))
   

    past_dict={'Date_Time':Date_Time,'Open':O,'High':H,'Low':L,'Close':C,'Volume':Volume,'C_O':C_minus_O,'H_L':H_minus_L}
    df=pd.DataFrame(past_dict)
    df=df.tail(far_back)
    
    
    return df
x=past_data(jpy_pairs[3],300,150)
x.tail()


def start_of_hour(pair):
    params={"count":1,"granularity":"H1"}
    r = instruments.InstrumentsCandles(instrument=pair,params=params)
    a=api.request(r)
    hr_start=float(a['candles'][-1]['mid']['o'])
    
    return hr_start



def reversal_creation(num_candles,np_data,_list):
    for i in range(np_data.shape[0]):
        if i<num_candles:
            _list.append(0)
        elif np.nanmin(np_data[i-num_candles:i+num_candles,4])==np_data[i,4]:
            _list.append(1)
        elif np.nanmax(np_data[i-num_candles:i+num_candles,4])==np_data[i,4]:
            _list.append(2)
        else:
            _list.append(0)
    return _list




def closest_reversal(rev_data,hour_val):
    support_diff=1000
    resistance_diff=1000
    support=0
    resistance=0
    support_wick=0
    resistance_wick=0
    
    for i in range(rev_data.shape[0]):
        if rev_data[i,8]==1 and (hour_val>=rev_data[i,4]) and (support_diff>=(hour_val-rev_data[i,4])):
            support_diff=hour_val-rev_data[i,4]
            support=rev_data[i,4]
            support_wick=rev_data[i,3]
            
        
        elif rev_data[i,8]==2 and (hour_val<=rev_data[i,4]) and(resistance_diff>=(rev_data[i,4]-hour_val)):
            resistance_diff=rev_data[i,4]
            resistance=rev_data[i,4]
            resistance=rev_data[i,2]
            
            
        
        else:
            continue
     
    return support,resistance,support_wick,resistance_wick

def check_price(pair):
    bid_ask=[]
    params={"instruments":pair}
            
    r = pricing.PricingInfo(accountID=accountID, params=params)
    a=api.request(r)
    
    bid=float(a['prices'][0]['bids'][0]['price'])
    ask=float(a['prices'][0]['asks'][0]['price'])
    avg=(bid+ask)/2
    
    bid_ask.append(bid)
    bid_ask.append(ask)
    bid_ask.append(avg)
    
    
    
    return bid_ask
def pair_reversal_func(data,hour):
    pand_df=data
    reversal=[]
    num_p=pand_df.values
    reversal_creation(25,num_p,reversal)
    pand_df['reversal']=reversal
    num_p=pand_df.values
    s,r,s_w,r_w=closest_reversal(num_p,hour)
    
    return s,r,s_w,r_w

def account_value():
        
    api = oandapyV20.API(access_token=access_token)
    r = accounts.AccountDetails(accountID)
    acc_details=api.request(r)
    
    balance=float(acc_details['account']['balance'])
    margin_available=float(acc_details['account']['marginAvailable'])
    open_trades=float(acc_details['account']['openTradeCount'])
    
    return balance,margin_available,open_trades
account_value()

In [ ]:
params={}
r = instruments.InstrumentsOrderBook(instrument='EUR_USD',params=params)
p = instruments.InstrumentsPositionBook(instrument='EUR_USD',params=params)

accountID='101-001-10831130-001'
access_token='45cc721fe40759fb0b76d015456ba8b8-27c86888ddbab88249c3914ba1ff2a01'

api = API(access_token)
api.request(r)

res=r.response

In [ ]:
price=pos['positionBook']['price']

In [ ]:
api.request(p)
pos=p.response


In [ ]:
s=time.time()
for i in range(100):
    api.request(p)
    pos=p.response
e=time.time()
e-s

In [ ]:
c=pd.DataFrame(pos['positionBook']['buckets'])
c['price']=c.price.astype(float)
pl=float(price)+.005
mi=float(price)-.005
print(price)
c[(c['price'] > mi) & (c['price'] < pl)]

In [ ]:
import time
s=time.time()
for i in range(1000):
    a=pd.DataFrame(res['orderBook']['buckets'])
    a['price']=a.price.astype(float)
    pl=float(price)+.001
    mi=float(price)-.001
    b=a[(a['price'] > mi) & (a['price'] < pl)]
e=time.time()
e-s

In [ ]:
price=res['orderBook']['price']

a=pd.DataFrame(res['orderBook']['buckets'])
a['price']=a.price.astype(float)
pl=float(price)+.007
mi=float(price)-.007
print(price)
a[(a['price'] > mi) & (a['price'] < pl)]

In [ ]:
(a['price'] > mi)